# Usa scikit-learn y una librería personalizada para predecir temperatura con `ibm-watson-machine-learning`

Este notebook contiene pasos y código para entrenar un modelo Scikit-Learn que usa un transformador personalizado y desplegarlo con el servicio de Watson Machine Learning. Una vez que el modelo se haya entrenado, este notebook contiene los pasos para persistir el modelo y su transformador en el repositorio de Watson Machine Learning, además de poder desplegarlo y evaluarlo con el cliente de Python de Watson Machine Learning.

En este notebook utilizamos un dataset de GNFUV que contiene lecturas de senores móviles que recolectan datos sobre humedad y temperatura por vehículos en un banco de pruebas en Atenas para entrenar a nuestro modelo para predecir la temperatura.

Se recomienda tener familiaridad con Python. Este notebook utiliza Python-3.7 y scikit-learn-0.23.1

## Objetivos de aprendizaje

Los objetivos de aprendizaje de este notebook son los siguientes:

- Entrenar un modelo con un transformador personalizado
- Persistir el transformador y el modelo dentro del repositorio de Watson Machine Learning.
- Desplegar el modelo utilizando Watson Machine Learning.
- Realizar predicciones utilizando el modelo desplegado.

## Contenidos
1.	[Preparación del ambiente](#setup)
2.	[Instalación de la librería que contiene el transformador](#install_lib)
3.  [Prepara los datos](#load)
4.	[Entrena el modelo](#train)
5.	[Guarda el modelo](#persistence)
6.	[Despliega y evalua el modelo](#deploy)
7.	[Resumen y próximos pasos](#summary)


<a id="setup"></a>
## 1. Preparación del ambiente

### Conexión con WML

Autentícate con el servicio de Watson Machine Learning en IBM Cloud Pak for Data.

In [1]:
api_key = 'PON AQUÍ TU API KEY DE IBM CLOUD'
location = 'PON AQUÍ LA UBICACIÖN DE TU SERVICIO DE WML'

In [2]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

### Instalar e importar el paquete `ibm-watson-machine-learning`
**Nota:** La documentación de `ibm-watson-machine-learning` puede ser encontrada <a href="http://ibm-wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">aquí</a>.

In [ ]:
!pip install -U ibm-watson-machine-learning

In [3]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

In [4]:
space_id = 'PON AQUÍ EL ID DE TU ESPACIO DE DESPLIEGUE'

Puedes utilizar el método `list` para ver todos los espacios de este servicio.

In [ ]:
client.spaces.list(limit=10)

Para ser capaces de interactuar con los recursos disponibles en Watson Machine Learning necesitas configurar el **espacio** el cuál vas a utilizar.

In [4]:
client.set.default_space(space_id)

'SUCCESS'

<a id="install_lib"></a>
## 2. Instalación de la librería que contiene el transformador

La librería `linalgnorm-0.1.zip` es un paquete de Python que contiene la implementación de un transformador para Scikit-Learn, el `LNormalizer`. <br>
En esta sección, vamos a descargar e instalar la librería en el entorno actual del notebook.

In [5]:
!wget https://github.com/IBM/watson-machine-learning-samples/raw/master/cpd3.5/libraries/linalgnorm-0.1.zip --output-document=linalgnorm-0.1.zip

--2020-12-08 12:45:08--  https://github.com/IBM/watson-machine-learning-samples/raw/master/cpd3.5/libraries/linalgnorm-0.1.zip
Resolving github.com... 140.82.121.3
Connecting to github.com|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cpd3.5/libraries/linalgnorm-0.1.zip [following]
--2020-12-08 12:45:08--  https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cpd3.5/libraries/linalgnorm-0.1.zip
Resolving raw.githubusercontent.com... 151.101.192.133, 151.101.0.133, 151.101.64.133, ...
Connecting to raw.githubusercontent.com|151.101.192.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2550 (2.5K) [application/zip]
Saving to: 'linalgnorm-0.1.zip'

linalgnorm-0.1.zip  100%[===================>]   2.49K  --.-KB/s    in 0.009s  

2020-12-08 12:45:08 (280 KB/s) - 'linalgnorm-0.1.zip' saved [2550/2550]



Instala la librería descargada utilizando el comando `pip`

In [6]:
!pip install linalgnorm-0.1.zip

Processing ./linalgnorm-0.1.zip
  Created wheel for linalgnorm: filename=linalgnorm-0.1-py3-none-any.whl size=1670 sha256=5416b34c623f8502515a75d8f9de1f6fce41fe55cd31ab9ab87863e6f7f9df23
  Stored in directory: /Users/jansoltysik/Library/Caches/pip/wheels/78/00/7b/c263b6176f7c38c807f442edaa5f11a3e7a2cbcc5fa07b2673
Successfully built linalgnorm
  Attempting uninstall: linalgnorm
    Found existing installation: linalgnorm 0.1
    Uninstalling linalgnorm-0.1:
      Successfully uninstalled linalgnorm-0.1


<a id="load"></a>
## 3. Prepara los datos

Descarga los datos del repositorio de UCI - https://archive.ics.uci.edu/ml/machine-learning-databases/00452/GNFUV%20USV%20Dataset.zip

In [7]:
!rm -rf dataset
!mkdir dataset

In [8]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00452/GNFUV%20USV%20Dataset.zip --output-document=dataset/gnfuv_dataset.zip

--2020-12-08 12:45:12--  https://archive.ics.uci.edu/ml/machine-learning-databases/00452/GNFUV%20USV%20Dataset.zip
Resolving archive.ics.uci.edu... 128.195.10.252
Connecting to archive.ics.uci.edu|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 501978 (490K) [application/x-httpd-php]
Saving to: 'dataset/gnfuv_dataset.zip'

dataset/gnfuv_datas 100%[===================>] 490.21K   119KB/s    in 4.1s    

2020-12-08 12:45:17 (119 KB/s) - 'dataset/gnfuv_dataset.zip' saved [501978/501978]



In [9]:
!unzip dataset/gnfuv_dataset.zip -d dataset

Archive:  dataset/gnfuv_dataset.zip
  inflating: dataset/pi2/gnfuv-temp-exp1-55d487b85b-5g2xh_1.0.csv  
  inflating: dataset/pi3/gnfuv-temp-exp1-55d487b85b-2bl8b_1.0.csv  
  inflating: dataset/pi4/gnfuv-temp-exp1-55d487b85b-xcl97_1.0.csv  
  inflating: dataset/pi5/gnfuv-temp-exp1-55d487b85b-5ztk8_1.0.csv  
  inflating: dataset/README.pdf      


Crea un dataframe de pandas basado en el dataset recién descargado

In [10]:
import json
import pandas as pd
import numpy as np
import os
from datetime import datetime
from json import JSONDecodeError

In [11]:
home_dir = './dataset'
pi_dirs = os.listdir(home_dir)

data_list = []
base_time = None
columns = None

for pi_dir in pi_dirs:
    if 'pi' not in pi_dir:
        continue
    curr_dir = os.path.join(home_dir, pi_dir)
    data_file = os.path.join(curr_dir, os.listdir(curr_dir)[0])
    with open(data_file, 'r') as f:
        line = f.readline().strip().replace("'", '"')
        while line != '':
            try:
                input_json = json.loads(line)
                sensor_datetime = datetime.fromtimestamp(input_json['time'])
                if base_time is None:
                    base_time = datetime(sensor_datetime.year, sensor_datetime.month, sensor_datetime.day, 0, 0, 0, 0)
                input_json['time'] = (sensor_datetime - base_time).seconds
                data_list.append(list(input_json.values()))
                if columns is None:
                    columns = list(input_json.keys())
            except JSONDecodeError as je:
                pass
            line = f.readline().strip().replace("'", '"')

data_df = pd.DataFrame(data_list, columns=columns)

In [12]:
data_df.head()

,device,humidity,temperature,experiment,time
0,gnfuv-temp-exp1-55d487b85b-5g2xh,21.0,40.0,1.0,69557
1,gnfuv-temp-exp1-55d487b85b-5g2xh,21.0,40.0,1.0,69571
2,gnfuv-temp-exp1-55d487b85b-5g2xh,21.0,40.0,1.0,69577
3,gnfuv-temp-exp1-55d487b85b-5g2xh,21.0,40.0,1.0,69583
4,gnfuv-temp-exp1-55d487b85b-5g2xh,22.0,40.0,1.0,69589


Crea grupos de datos tanto de entrenamiento como de pruebas del dataset descargado de GNFUV-USV

In [13]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

Y = data_df['temperature']
X = data_df.drop('temperature', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=143)

<a id="train"></a>
## 4. Entrena el modelo

En esta sección utilizaras el transformador personalizado como una etapa de entrenamiento del `Pipeline` de Scikit-Learn.

#### Importa el transformador personalizado
Aquí vamos a imporar el transformador que fue definido en `linalgnorm-0.1.zip` y crearemos una instancia de él que será utilizado como etapa en `sklearn.Pipeline`

In [14]:
from linalg_norm.sklearn_transformers import LNormalizer

In [15]:
lnorm_transf = LNormalizer()

Importación otros objetos requeridos para entrenar al modelo

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

Ahora tú puedes crear una `Pipeline` con un transformador personalizado como una de las etapas de entrenamiento del modelo

In [17]:
skl_pipeline = Pipeline(steps=[('normalizer', lnorm_transf), ('regression_estimator', LinearRegression())])
skl_pipeline.fit(X_train.loc[:, ['time', 'humidity']].values, y_train)

Pipeline(steps=[('normalizer', LNormalizer()),
                ('regression_estimator', LinearRegression())])

In [18]:
y_pred = skl_pipeline.predict(X_test.loc[:, ['time', 'humidity']].values)
rmse = np.mean((np.round(y_pred) - y_test.values)**2)**0.5
print('RMSE: {}'.format(rmse))

RMSE: 2.213758431322581


<a id="persistence"></a>
## 5. Guarda el modelo
En esta sección con la ayuda del SDK `ibm-watson_machine_learning` vas a:
- Guardar la librería `linalgnorm-0.1.zip` en el repositorio de Watson Machine Learning creando un recurso de él mismo
- Crear un recurso de Especificación de Software para configurar el despliegue y el runtime del modelo
- Enlazar esta Especificación de Software con el modelo y guardarlo en el repositorio de Watson Machine Learning

### Crea la extensión del paquete

Define los metadatos requeridos para crear un recurso y guardar la librería en Watson Machine Learning. <br>

El valor para `file_path` en `client.package_extensions.LibraryMetaNames.store()` contiene el nombre de la librería que debe ser subida a WML.

**Nota:** También puedes utilizar un archivo de configuración de conda (`yaml`) como entrada. En ese caso se debe cambiar el campo `TYPE` a `conda_yml` y `file_path` debería apuntar al archivo yaml.
```
client.package_extensions.ConfigurationMetaNames.TYPE = "conda_yml"
```

In [19]:
meta_prop_pkg_extn = {
    client.package_extensions.ConfigurationMetaNames.NAME: "K_Linag_norm_skl",
    client.package_extensions.ConfigurationMetaNames.DESCRIPTION: "Pkg extension for custom lib",
    client.package_extensions.ConfigurationMetaNames.TYPE: "pip_zip"
}

pkg_extn_details = client.package_extensions.store(meta_props=meta_prop_pkg_extn, file_path="linalgnorm-0.1.zip")
pkg_extn_uid = client.package_extensions.get_uid(pkg_extn_details)
pkg_extn_url = client.package_extensions.get_href(pkg_extn_details)

Creating package extensions
SUCCESS


Despliega los detalles del paquete que fue creado en la celda anterior.

In [20]:
details = client.package_extensions.get_details(pkg_extn_uid)

### Crea la especificación de software y agrega la librería personalizada

Define los metadatos requeridos para crear un recurso de especificación de software y enlazarlo con el paquete. Este recurso será utilizado para configurar el despliegue del entorno para un modelo.

In [21]:
client.software_specifications.ConfigurationMetaNames.show()

---------------------------  ----  --------  --------------------------------
META_PROP NAME               TYPE  REQUIRED  SCHEMA
NAME                         str   Y
DESCRIPTION                  str   N
PACKAGE_EXTENSIONS           list  N
SOFTWARE_CONFIGURATION       dict  N         {'platform(required)': 'string'}
BASE_SOFTWARE_SPECIFICATION  dict  Y
---------------------------  ----  --------  --------------------------------


#### Lista de las especificaciones base

In [22]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
tensorflow_1.15-py3.6          2b73a275-7cbf-420b-a912-eae7f436e0bc  base
pytorch_1.2-py3.6              2c8ef57d-2687-4b7d-acce-01f94976dac1  base
spark-mllib_2.3                2e51f70

#### Selecciona una especificación base por la cuál nos extenderemos

In [23]:
base_sw_spec_uid = client.software_specifications.get_uid_by_name("default_py3.7")

#### Define una espeficiación nueva basada en una base y una librería personalizada

In [24]:
meta_prop_sw_spec = {
    client.software_specifications.ConfigurationMetaNames.NAME: "linalgnorm-0.1",
    client.software_specifications.ConfigurationMetaNames.DESCRIPTION: "Software specification for linalgnorm-0.1",
    client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {"guid": base_sw_spec_uid}
}

sw_spec_details = client.software_specifications.store(meta_props=meta_prop_sw_spec)
sw_spec_uid = client.software_specifications.get_uid(sw_spec_details)


client.software_specifications.add_package_extension(sw_spec_uid, pkg_extn_uid)

SUCCESS


'SUCCESS'

### Guarda el modelo

Define la metadata para guardar el modelo entrenado en el repositorio de WML junto a la información requerida para el entorno del modelo.

La propiedad `client.repository.ModelMetaNames.SOFTWARE_SPEC_UID` es usada para especificar la especificación de software con la cuál será asociada el modelo.

In [25]:
model_props = {
    client.repository.ModelMetaNames.NAME: "Temp prediction model with custom lib",
    client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.23',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid
    
}

Guarda el modelo en el repositorio de WML y despliega su metadata guardada.

In [26]:
published_model = client.repository.store_model(model=skl_pipeline, meta_props=model_props)

In [27]:
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

{
  "entity": {
    "software_spec": {
      "id": "dae27ebc-97b7-450e-bd27-60bd1e5ca198",
      "name": "linalgnorm-0.1"
    },
    "type": "scikit-learn_0.23"
  },
  "metadata": {
    "created_at": "2020-12-08T11:45:55.651Z",
    "id": "a6a27638-71ee-493c-b6b6-d8488958b974",
    "modified_at": "2020-12-08T11:45:57.475Z",
    "name": "Temp prediction model with custom lib",
    "owner": "1000330999",
    "space_id": "83b00166-9047-4159-b777-83dcb498e7ab"
  },
  "system": {
    "warnings": []
  }
}


<a id="deploy"></a>
## 6. Despliega y evalua el modelo

En esta sección desplegarás tu modelo guardado que utiliza un transformador personalizado y harás predicciones. Utilizarás el cliente de WML para realizar estas tareas.

### Despliega el modelo

In [28]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of custom lib model",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)



#######################################################################################

Synchronous deployment creation for uid: 'a6a27638-71ee-493c-b6b6-d8488958b974' started

#######################################################################################


initializing.
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='0d94523d-a2e7-4d75-b753-648fae333747'
------------------------------------------------------------------------------------------------




### Predice utilizando el modelo desplegado

**Nota:** Aquí usamos el `uid` obtenido al desplegar el modelo. En la siguiente sección te enseñaremos como obtener la URL de la instancia de Watson Machine Learning.

In [29]:
deployment_uid = client.deployments.get_uid(created_deployment)

Ahora puedes imprimir el servicio que expuso el despliegue de tu modelo!

In [30]:
scoring_endpoint = client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

https://wmlgmc-cpd-wmlgmc.apps.wmlautoai.cp.fyre.ibm.com/ml/v4/deployments/0d94523d-a2e7-4d75-b753-648fae333747/predictions


Prepara el paquete para la predicción. El paquete contiene las entradas para las cuáles se quiere realizar una predicción.

In [31]:
scoring_payload = {
    "input_data": [{
        'fields': ["time", "humidity"],
        'values': [[79863, 47]]}]
}

Ejecuta el método para performar predicciones y desplegar sus resultados

In [32]:
predictions = client.deployments.score(deployment_uid, scoring_payload)

In [33]:
print(json.dumps(predictions, indent=2))

{
  "predictions": [
    {
      "fields": [
        "prediction"
      ],
      "values": [
        [
          14.629242312262988
        ]
      ]
    }
  ]
}


<a id="summary"></a>

## 7. Resumen y próximos pasos
Acabas de terminar por completo este notebook!

Aprendiste como utilizar un modelo de scikit-learn con un transformador personalizado en el servicio de Watson Machine Learning para desplegar y evaluar.

Mira nuestra [Documentación Online](https://dataplatform.cloud.ibm.com/docs/content/analyze-data/wml-setup.html) para más muestras, tutoriales, documentación, guías y artículos!

## Autores
**Krishnamurthy Arthanarisamy**, es un líder técnico senior en el equipo de IBM Watson Machine Learning. Krishna trabaja en desarrollar servicios en la nube que atiende las diferentes etapas en el ciclo de vida de modelado en machine learning y deep learning

**Lukasz Cmielowski**, PhD, es un Arquitecto de Software y Data Scientist en IBM.

Traducciones y modificaciones hechas por **José Paz**

Copyright © 2020, 2021 IBM. This notebook and its source code are released under the terms of the MIT License.